<a href="https://colab.research.google.com/github/bitsym/InformerforCurve/blob/main/CurvesOT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, MultiHeadAttention, Input
from tensorflow.keras.models import Model
import numpy as np

class PositionalEncoding(Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)[tf.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]
class TransformerEncoder(Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerEncoder, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
def build_transformer(num_features, sequence_length, d_model, num_heads, ff_dim, num_layers, rate=0.1):
    inputs = Input(shape=(sequence_length, num_features))
    # Map the input features to the model dimension
    x = Dense(d_model, activation='relu')(inputs)
    # Add positional encodings
    x = PositionalEncoding(sequence_length, d_model)(x)
    for _ in range(num_layers):
        x = TransformerEncoder(d_model, num_heads, ff_dim, rate)(x)
    outputs = Dense(1, activation='linear')(x)  # Predicting 'OT1'
    model = Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the CSV file
data = pd.read_csv('CurvesT3.csv')

# Select features and target
features = data[['LOCATION', 'X SERIAL', 'Y SERIAL', 'Z SERIAL']]
print(data)

     LOCATION   X SERIAL   Y SERIAL   Z SERIAL      OT3 
0    0.000000   0.000000   0.000000   0.000000 -0.242587
1    0.050267  -2.422151  -1.368757   3.929735 -0.326288
2    0.100688  -5.581641  -3.148854   7.076572 -0.425911
3    0.150950  -9.442119  -5.338873   8.929961 -0.475210
4    0.200955 -13.505708  -7.671132  10.064515 -0.488859
..        ...        ...        ...        ...       ...
331  0.801434 -64.007807 -36.688473  -3.398382 -0.450358
332  0.851568 -67.827846 -38.824129  -5.649999 -0.434834
333  0.901326 -71.574625 -40.908549  -8.068927 -0.439284
334  0.950659 -75.440764 -43.085383 -10.197360 -0.464620
335  1.000000 -79.496033 -45.445277 -11.682654 -0.482181

[336 rows x 5 columns]


In [ ]:
target = data[]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the CSV file
data = pd.read_csv('CurvesT3.csv')

# Select features and target
features = data[['LOCATION', 'X SERIAL', 'Y SERIAL', 'Z SERIAL']]
target = data['OT3']

# Normalize the feature data
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Check the sequence creation logic
def create_sequences(features, target, sequence_length):
    X, y = [], []
    for i in range(0, len(features) - sequence_length + 1, sequence_length):  # Ensure no overlap if not desired
        X.append(features[i:(i + sequence_length)])
        y.append(target[i + sequence_length - 1])  # Adjust based on how you want to predict
    return np.array(X), np.array(y)

# Assuming sequence_length is correctly set and there's enough data
features_seq, target_seq = create_sequences(features_scaled, target, sequence_length=21)
print("Generated Features Shape:", features_seq.shape)
print("Generated Target Shape:", target_seq.shape)

# Generate sequences
sequence_length = 21
features_seq, target_seq = create_sequences(features_scaled, target, sequence_length)

# Split the data into training, validation, and testing sets
# Note: Adjust indices based on how you define training, validation, and testing
index_train = len(features_seq) - 2 * sequence_length  # Reserve the last 42 sequences
index_val = len(features_seq) - sequence_length  # Reserve the last 21 sequences

train_features = features_seq[:index_train]
train_target = target_seq[:index_train]

val_features = features_seq[index_train:index_val]
val_target = target_seq[index_train:index_val]

test_features = features_seq[index_val:]
test_target = target_seq[index_val:]

# Now you can proceed to define and train your model with train_features, val_features, and test_features
# Assuming the total number of sequences and the sequence length
total_sequences = features_seq.shape[0]
train_features = features_seq[:int(total_sequences * 0.8)]
train_target = target_seq[:int(total_sequences * 0.8)]
val_features = features_seq[int(total_sequences * 0.8):int(total_sequences * 0.9)]
val_target = target_seq[int(total_sequences * 0.8):int(total_sequences * 0.9)]


KeyError: 'OT3'

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Adjust model dimensions and architecture
d_model = 1024  # increased from 512
ff_dim = 4096   # increased from 2048
num_layers = 4  # increased from 2

model = build_transformer(num_features=4, sequence_length=21, d_model=d_model, num_heads=8, ff_dim=ff_dim, num_layers=num_layers)
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Add learning rate scheduler and early stopping
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    train_features, train_target,
    validation_data=(val_features, val_target),
    epochs=100,
    batch_size=32,
    callbacks=[lr_scheduler, early_stopping]
)



In [ ]:
import matplotlib.pyplot as plt

# Plot learning rate vs loss
plt.semilogx(history.history["lr"], history.history["loss"])
plt.xlabel("Learning Rate")
plt.ylabel("Loss")
plt.title("Learning Rate vs. Loss")
plt.show()


In [ ]:
total_sequences = len(features_seq)
print("Total sequences available:", total_sequences)
# Make predictions on the test dataset
test_predictions = model.predict(test_features)
test_pre = test_predictions[0,:]
print(test_pre)
testPre = test_pre[:,0]
print(testPre)

In [ ]:
print(test_target)
print(target)
testY = target[-21:]
print(testY)
print(testY)
print(len(testY))
testY = testY.to_numpy()


In [ ]:
import matplotlib.pyplot as plt

# Assuming test_target is correctly aligned with test_predictions
# plt.figure(figsize=(12, 6))
plt.plot(testY, label='Actual Values', linestyle='-', color='blue')
plt.plot(testPre, label='Predicted Values', linestyle='--', color='red')
plt.title('Comparison of Actual and Predicted Values')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd

# Create a DataFrame with both testY and testPre
data = {
    'Actual Values': testY,
    'Predicted Values': testPre
}
df = pd.DataFrame(data)
# Saving the DataFrame to a CSV file
df.to_csv('predictions_comparison_OT3.csv', index=False)
# Saving each vector to its own CSV file
df_actual = pd.DataFrame(testY, columns=['Actual Values'])
df_predicted = pd.DataFrame(testPre, columns=['Predicted Values'])

df_actual.to_csv('actual_valuesOT3.csv', index=False)
df_predicted.to_csv('predicted_valuesOT3.csv', index=False)
